In [49]:
import zarr
import numpy as np
import spatialdata as sd

/Users/mkeller/software/miniconda3/envs/point-cloud-env/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/mkeller/software/miniconda3/envs/point-cloud-env/lib/python3.11/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [48]:
# Download merfish spatialdata example from https://spatialdata.scverse.org/en/latest/tutorials/notebooks/notebooks/examples/technology_merfish.html

In [108]:
merfish_zarr_path = "./data/data.zarr"

In [109]:
merfish_sdata = sd.read_zarr(merfish_zarr_path)
merfish_sdata

SpatialData object with:
├── Images
│     └── 'rasterized': SpatialImage[cyx] (1, 522, 575)
├── Points
│     └── 'single_molecule': DataFrame with shape: (3714642, 3) (2D points)
├── Shapes
│     ├── 'anatomical': GeoDataFrame shape: (6, 1) (2D shapes)
│     └── 'cells': GeoDataFrame shape: (2399, 2) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 2399 × 268
    obs: 'cell_id', 'region'
    uns: 'spatialdata_attrs': AnnData (2399, 268)
with coordinate systems:
▸ 'global', with elements:
        rasterized (Images), single_molecule (Points), anatomical (Shapes), cells (Shapes)

In [110]:
df = merfish_sdata.points['single_molecule']

In [111]:
x_min = df['x'].min().compute()
x_max = df['x'].max().compute()

y_min = df['y'].min().compute()
y_max = df['y'].max().compute()

In [112]:
c_cats = df['cell_type'].unique().compute().tolist()
c_cat_to_index = dict([(c_cat, c_cats.index(c_cat)) for c_cat in c_cats])
c_cat_to_index

{'outside_VISp': 0,
 'VISp_wm': 1,
 'VISp_VI': 2,
 'VISp_V': 3,
 'VISp': 4,
 'VISp_IV': 5,
 'VISp_II/III': 6,
 'VISp_I': 7}

In [113]:
x_min, x_max, y_min, y_max

(1154.3634, 3171.979, 4548.483, 6565.997)

In [114]:
scale_factor = 4

pixel_grid_shape = (np.ceil(x_max - x_min) * scale_factor, np.ceil(y_max - y_min) * scale_factor)
pixel_grid_shape

(8072.0, 8072.0)

In [115]:
z = zarr.zeros(pixel_grid_shape)
chunk_shape = z.chunks

In [116]:
z_x = zarr.zeros(pixel_grid_shape)
z_y = zarr.zeros(pixel_grid_shape)
z_c = zarr.zeros(pixel_grid_shape)

In [118]:
df['x'] = ((df['x'] - x_min) / (x_max - x_min)) * pixel_grid_shape[0]
df['y'] = ((df['y'] - y_min) / (y_max - y_min)) * pixel_grid_shape[1]

In [119]:
df.head()

,x,y,cell_type
0,497.376984,5890.054688,outside_VISp
1,690.559265,5903.028320,outside_VISp
2,550.758728,6044.871582,outside_VISp
3,586.203003,6198.705078,outside_VISp
4,312.246002,6219.102539,outside_VISp


In [124]:
chunk_df['x'].values.compute()

array([], dtype=float32)

In [127]:
for x_chunk_i in range(int(np.ceil(pixel_grid_shape[0] / chunk_shape[0]))):
    for y_chunk_i in range(int(np.ceil(pixel_grid_shape[1] / chunk_shape[1]))):
        print(x_chunk_i, y_chunk_i)
        x_offset = x_chunk_i * chunk_shape[0]
        y_offset = y_chunk_i * chunk_shape[1]
        x_chunk_max = (x_chunk_i + 1) * chunk_shape[0]
        y_chunk_max = (y_chunk_i + 1) * chunk_shape[0]

        chunk_df = df.loc[(df['x'] >= x_offset) & (df['x'] < x_chunk_max) & (df['y'] >= y_offset) & (df['y'] < y_chunk_max)]
        
        chunk_x_vals = chunk_df['x'].values.compute()
        chunk_y_vals = chunk_df['y'].values.compute()
        #chunk_c_vals = chunk_df['cell_type'].values.astype(str).compute()

        
        print(chunk_df.shape[0].compute())

0 0
0
0 1
0
0 2
0
0 3
0
0 4
0
0 5
0
0 6
0
0 7
0
0 8
0
0 9
0
0 10
0
0 11
3079
0 12
26279
0 13
8519
0 14
0
0 15
0
1 0
0
1 1
0
1 2
0
1 3
0
1 4
0
1 5
0
1 6
0
1 7
0
1 8
473
1 9
2492
1 10
11391
1 11
14189
1 12
29805
1 13
6249
1 14
0
1 15
0
2 0
0
2 1
0
2 2
0
2 3
0
2 4
0
2 5
0
2 6
0
2 7
0
2 8
840
2 9
1122
2 10
7882
2 11
6980
2 12
11460
2 13
3668
2 14
0
2 15
0
3 0
0
3 1
0
3 2
0
3 3
0
3 4
0
3 5
0
3 6
101
3 7
509
3 8
742
3 9
876
3 10
2085
3 11
10009
3 12
21876
3 13
19922
3 14
20644
3 15
456
4 0
0
4 1
0
4 2
0
4 3
114
4 4
516
4 5
92
4 6
290
4 7
1192
4 8
691
4 9
1363
4 10
16743
4 11
33726
4 12
36559
4 13
26143
4 14
22091
4 15
727
5 0
0
5 1
0
5 2
0
5 3
19932
5 4
30599
5 5
8085
5 6
2785
5 7
2152
5 8
2377
5 9
9064
5 10
34193
5 11
35685
5 12
34510
5 13
25718
5 14
22749
5 15
40897
6 0
0
6 1
861
6 2
8280
6 3
16016
6 4
29332
6 5
10912
6 6
6067
6 7
19883
6 8
23169
6 9
34180
6 10
32836
6 11
25627
6 12
28946
6 13
28514
6 14
20821
6 15
25526
7 0
0
7 1
8771
7 2
27864
7 3
6922
7 4
8899
7 5
20626
7 6
38546
7 7
36

KeyboardInterrupt: 